In [1]:
# open AI 와 LLM과의 차이점은 세부 사항에 있ek

# open AI는 gpt-3.5-turbo를 model로 사용하고 있는 것을 확인 할 수 있다.

In [2]:
#from langchain.llms.openai import OpenAI

# 주로 사용할것 chatgpt-3.5-turbo
from langchain.chat_models import ChatOpenAI 

In [3]:
# OpenAI(openai_api_key = "자신의 api 키")를 넣을 수도 있다
# (만약 .env 파일 안에 OPENAI_API_KEY="자신의 api 키"를 안 넣어 놨으면)
chat = ChatOpenAI()

# llm = OpenAI()
# a = llm.predict("How many planets are there?")
# InvalidRequestError: The model `text-davinci-003` has been deprecated 에러가 뜬다.
# 사용중인 모델인 text-davinci-003은 더 이상 사용되지 않는다는 것

b = chat.predict("How many planets are there?")

b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage


# chat model 만들기
chat = ChatOpenAI(temperature=0.1)

#
messages = [
    SystemMessage(content = "You are a geography expert. And you only reply in korean",),
    AIMessage(content="안녕하세요!!!!"),
    HumanMessage(content="hi!  what is the distance between Mexico and Thailand. Also, what is your name?",),
]


# 단순히 문자열들을 predict 하는 것 대신에 
# predict_messages를 사용하고 message의 list를 넘겨준다.
chat.predict_messages(messages)

AIMessage(content='멕시코와 태국 사이의 거리는 대략 16,000km입니다. 제 이름은 비밀입니다. 어떤 도움이 필요하십니까?')

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")

prompt=template.format(country_a = "Mexico",country_b="Korea")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and Korea is approximately 8,900 miles (14,300 kilometers) when measured in a straight line.'

In [8]:
template = ChatPromptTemplate.from_messages([
        ("system","You are a geography expert. And you only reply in {language}."),
        ("ai","안녕 내 이름은 {name}야"),
        ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
prompt=template.format_messages(language="Korea",name="chat-gpt",country_a = "Mexico",country_b="Korea")
chat.predict_messages(prompt)

AIMessage(content='멕시코와 한국 사이의 거리는 약 12,000km입니다. 내 이름은 chatgpt야.')

In [9]:
# 응답을 list로 변환시키는 것을 만들어 볼 예정

# OutputParser는  parse라는 메서드를 꼭 구현해야 한다.
# strip은 text의 앞뒤 공백을 잘라준다.


from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        itmes = text.strip().split(",")
        return list(map(str.strip,itmes))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [15]:
# prompt template을 포맷팅 한 후
template = ChatPromptTemplate.from_messages([
    ("system","""You are a list generating machine. Everything you are asked will be 
     answered with a comma separated list of max {max_items}. Do NOT reply with anything else."""),
     ("human","{question}")
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the color?"
)

result = chat.predict_messages(prompt)

# output Parser
p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [18]:
# LangChain Expression Language(표준언어)

# 재료 : chat model, outputParser class, template

# chain이라는 것은 기본적으로 모든 요소를 합쳐주는 역할을 한다.
# 합쳐진 요소들은 하나의 chain으로 실행 -> 하나하나 순서대로 result를 반환할때까지

# 내부에서 .format_messages를 호출 -> chat.predict 호출 -> parse 호출
chain = template | chat | CommaOutputParser()

# invoke 메서드의 입력값으로는 dictionary 타입이 들어가야된다.

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']